# Mouvements fluides du «Hero»

[Vidéo d'accompagnement](https://vimeo.com/506185892)

## Origine du problème

Pour comprendre pourquoi notre héro bouge «mal», copier et exécuter le script qui suit:

```python
from tkinter import Tk

# deux gestionnaires d'événements
def appui(evt):
    print(f'APPUI  = touche: {evt.keysym}, temps: {evt.time}')

def relache(evt):
    print(f'RELACHE= touche: {evt.keysym}, temps: {evt.time}')

fen = Tk()
fen.bind('<KeyPress>', appui)
fen.bind('<KeyRelease>', relache)
fen.mainloop()
```

On obtient par exemple (en enfonçant la touche a plus d'une demi-seconde):

    ...
    APPUI  = touche: a, temps: 872593933
    RELACHE= touche: a, temps: 872594433
    APPUI  = touche: a, temps: 872594433
    RELACHE= touche: a, temps: 872594471
    APPUI  = touche: a, temps: 872594471
    RELACHE= touche: a, temps: 872594504
    ...

Cela vient du clavier lui-même qui est normalement utilisé pour saisir du texte:

> si on garde une touche enfoncée, la lettre de la touche est «répétée» *automatiquement* comme si l'utilisateur tapotait sur la touche à un rythme très rapide.

Ainsi, lorsqu'on appuie sur une touche sans la relâcher, au bout d'une demi-seconde (500ms), le clavier génère des événements `KeyRelease`-`KeyPress` (dans cet ordre et simultanément) qui se répètent (environ 30 fois par seconde) tant qu'on ne relâche pas la touche.

Alors que nous, on souhaite effectuer des déplacements réguliers *seulement si la touche est enfoncée*.

## Solution

L'idée principale est de suivre les événements `KeyPress` et `KeyRelease` et de marquer l'état des touches intéressantes - *press* à True seulement si la touche est enfoncée.

Il va donc falloir détecter les «faux» `KeyPress`/`KeyRelease` en utilisant le fait que `evt.time` a la même valeur pour les deux s'ils sont faux. 

Pour y parvenir, nous allons créer deux classes:
- `Touche`: qui sert à représenter une touche particulière, son état (enfoncée ou non?) et l'action associée.

- `Clavier`: sert à représenter le clavier qu'on va scruter pour des touches déclarée via:
    - `reagir(nom_touche, action, dt)`: lorsque `nom_touche` est enfoncée, on souhaite répéter `action` toutes les `dt` ms.

Pour utiliser tout cela, il nous suffira d'écrire quelque chose comme:
```python
# l'action gauche est répétée toutes les 5Oms si la touche «flèche gauche» est enfoncée
clavier.reagir('left', gauche, 50)
# l'action tirer est déclenchée une seule fois lorsque la touche space est enfoncée.
clavier.reagir('space', tirer, 0)
```

où `clavier` est un objet de type `Clavier`.

### module clavier.py

*Note*: ce module est réutilisable dans n'importe quel projet de jeu avec tkinter.

Copier-coller le code qui suit dans un fichier *clavier.py*. Lire les commentaires si vous souhaitez comprendre l'«astuce».

```python
class Touche:
    def __init__(self, fen, action, dt):
        """action est une fonction sans argument qui sera appelée
        toutes les dt millisecondes lorsque la touche est enfoncée.
        Si dt = 0, l'action n'a lieu qu'une seule fois (elle n'est pas répétée)
        """
        self.fen = fen
        self.action = action
        self.dt = dt

        self.press = False # enfoncée ou non?

        self.time = 0 # horodatage pour reconnaître les faux relâchement (voir Clavier)

    def appui(self):
        self.press = True
        self.__agir() # on déclenche l'action à répéter toutes les dt millisecondes

    def relache(self):
        self.press = False

    def __agir(self):
        if not self.press: # touche relâchée: on arrête!
            return
        self.action()
        if self.dt == 0: # on a déjà agit donc on arrête
            return
        self.fen.after(self.dt, self.__agir)

class Clavier:

    def __init__(self, fen):
        # un dictionnaire pour
        self.touches = {} # forme {"nom_touche": obj de type Touche}
        self.fen = fen

        self.fen.bind('<KeyPress>', self.__appui)
        self.fen.bind('<KeyRelease>', self.__relache)

    def reagir(self, nom_touche, action, dt=40):
        # on crée une touche et on la mémorise
        self.touches[nom_touche] = Touche(self.fen, action, dt)
    
    def fin_reagir(self, nom_touche):
        del self.touches[nom_touche]

    def __appui(self, evt):
        # on récupère le nom de la touche:
        nom_touche = evt.keysym
        # si cette touche n'est pas enregistrée (voir reagir), on ne fait rien
        if not nom_touche in self.touches:
            return
        # on récupère l'objet touche
        touche = self.touches[nom_touche]
        # et on «marque» la touche avec l'horodatage de l'événement
        touche.time = evt.time
        # si la touche est déjà enfoncée, rien à faire de plus
        if touche.press:
            return
        # autrement on prévient la touche
        touche.appui()

    def __relache(self, evt):
        nom_touche = evt.keysym
        if not nom_touche in self.touches:
            return
        touche = self.touches[nom_touche]
        # si c'est un faux relâchement, la méthode __appui va être appelée immédiatement
        # on va vérifier cela après un délai très court de 5ms
        self.fen.after(5, lambda: self.__verif(touche, evt.time))

    def __verif(self, touche, time):
        if time == touche.time: # ainsi l'horodatage du relâchement et de l'appui sont les mêmes
            # c'est un faux: donc la touche est toujours enfoncée
            return
        # autrement, cela signifie que c'est un vrai relâchement et on prévient la touche.
        touche.relache()
```

Il ne reste plus qu'à adapter un peu:

1. dans *scene.py*, importer la classe `Clavier` puis ajouter `self.clavier = Clavier(fen)`.

2. dans la classe `Acteur`, adapter ...

```python
...
def reagir(self, nom_touche, action, delai=50):
    self.scene.clavier.reagir(nom_touche, action, delai)

def fin_reagir(self, nom_touche):
    self.scene.clavier.fin_reagir(nom_touche)
...
```

3. dans `Hero`, adapter aussi:
   
```python
    ...
    self.reagir('Left', self.gauche)
    self.reagir('Right', self.droite)
    self.reagir('space', self.tirer, delai=0)

def supprimer(self):
    super().supprimer()
    self.fin_reagir('space')
    self.fin_reagir('Right')
    self.fin_reagir('Left')
...
```


## Fichier principal *main.py*

Enfin, Pour lancer le jeu plus agréablement, créer un fichier principal *main.py*:

```python
from scene import *
from hero import Hero
from envahisseurs import EnvahisseurMere
from meute import Meute

Meute(scene, 5, 5)
EnvahisseurMere(scene)
Hero(scene)

fen.mainloop()
```

## Conclusion

Même s'il reste beaucoup à faire pour arriver à un jeu complet, ce TP vous aura permis d'acquérir des bases sur:

- tkinter et son widget `Canvas`,
- le découpage du code en **module**,
- la **séparation** entre *définition* d'une part, *test* d'autre part,
- la création et la gestion de nombreux objets grâce à la notion de **classe**,
- la *réutilisation* et/ou *spécialisation* du code grâce à la notion d'**héritage**,
- l'organisation des objets en grille grâce à la notion de **tableau**.

Notamment, vous devriez pouvoir comprendre comment le code du *space_invaders_final* situé dans le répertoire *archives* est organisé.